In [257]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [4]:
from urllib.request import urlopen
from urllib.request import Request
import urllib3
import json
from IPython.core.display import display, HTML
import re
from bs4 import BeautifulSoup

http = urllib3.PoolManager()

url="https://helpx.adobe.com/security/products/acrobat.html"

r = http.request('GET',url)
a=re.findall('<a disablelinktracking=\"false\" href=\"(.*)\" data-disable-query',r.data.decode('utf-8'))

r = http.request('GET',a[0])  
a=re.findall('<table width="100%" cellspacing="0" cellpadding="1" border="1">(.*)</table>',r.data.decode('utf-8'),re.MULTILINE|re.DOTALL)

display(HTML("<h1>Adobe Acrobat Latest Vulnerability</h1>"))
print(url)
soup = BeautifulSoup(a[0],"html.parser")

soup.find('div').decompose()
#display(HTML("<table>"+a[0]+"</table></div>"))
display(HTML("<table>"+str(soup)+"</table></div>"))

MaxRetryError: HTTPSConnectionPool(host='helpx.adobe.com', port=443): Max retries exceeded with url: /security/products/acrobat.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000288D1FD8550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
#AMD
#too much js



In [365]:
from datetime import date
from IPython.core.display import display, HTML

today = date.today()
date_today = today.strftime("%Y+%b")

request=urlopen("https://tools.cisco.com/security/center/publicationService.x?cves=&identifiers=&last_published_date="+date_today+"&limit=20&offset=0&publicationTypeIDs=1,3&resourceIDs=109810&securityImpactRatings=&sort=-last_published&title=")
cisco_data = request.read()
data = json.loads(cisco_data)
data = sorted(data, key=lambda k: k.get('severity', 0), reverse=False)
entries=len(data)
count=0
critical=0
high=0
medium=0

table="<table><tr><th>Identifer</th><th>Severity</th><th>Title</th><th>ciscoBugId</th><th>Last Published</th></tr>"

while count < entries:
#for identifier,title,version,firstPublished,lastPublished,workflowstatus,id,name,url,severity,workarounds,cwe,cve,ciscoBugId,status,summary,totalcount,realtedResource in data:
    table+="<tr><td>"+data[count]['identifier']+"</td><td>"+data[count]['severity']+"</td><td>"+data[count]['title']+"</td><td>"+(data[count]['ciscoBugId']).replace(',','\n')+"</td><td>"+data[count]['lastPublished'][:-18]+"</td></tr>"
    if data[count]['severity'] == 'Critical': critical+=1
    if data[count]['severity'] == 'High': high+=1
    if data[count]['severity'] == 'Medium': medium+=1
    count+=1
#print(x)

display(HTML("<h1>Cisco AnyConnect Monthly Vulnerability Feed</h1>"))
display(HTML(table+"</table>"))

Identifer,Severity,Title,ciscoBugId,Last Published
cisco-sa-20200108-anyconnect-hijack,Medium,Cisco AnyConnect Secure Mobility Client for Android Service Hijack Vulnerability,CSCvr67149,2020-01-08


In [419]:
import urllib3
import urllib.parse

today = date.today()
date_today = today.strftime("%m/%d/%Y")
r = http.request('GET',"https://portal.msrc.microsoft.com/api/security-guidance/en-us/summary?filterText=edge&fromPublishedDate=12%2F11%2F2019&isDescending=true&orderBy=publishedDate&pageNumber=1&pageSize=20&toPublishedDate="+date_today)
#print(r.data.decode('utf-8'))                     
data=json.loads(r.data.decode('utf-8'))

count=data["count"]

table="<table><tr><th>Date</th><th>Software</th><th>Urls</th></tr>"
for i in range(count):
  table+='<tr><td>'+data["details"][i]["publishedDate"][:10]+'</td><td>'+data["details"][i]["updateSoftwareList"][0]+'</td><td><a href="'+data["details"][i]["articleUrl1"]+'">'+data["details"][i]["articleUrl1"]+'</td></tr>'
table+="</table>"

display(HTML(table))


Date,Software,Urls
2020-01-28,Microsoft Edge (Chromium-based),https://review.docs.microsoft.com/en-us/DeployEdge/microsoft-edge-security-updates


[{"Name":"filter","Message":""}]


In [181]:
http = urllib3.PoolManager()
url="https://support.google.com/chrome/a/answer/7679408/"
r = http.request('GET',url)

a=re.findall('<td><a href=\"\#\d+\">(.*)</a>',r.data.decode('utf-8'))
b=re.findall('<td><a href=\"//storage(.*)\" target',r.data.decode('utf-8'))

display(HTML("<h1>Google Chrome Latest Version</h1>"))
display(HTML(a[0]+" <a href=\"https://storage"+b[0]+"\">Release PDF</a>"))

In [294]:

http = urllib3.PoolManager()
url="https://www.intel.com/content/www/us/en/security-center/default.html"
r = http.request('GET',url)

a=re.findall('<a href=(.*)\"',r.data.decode('utf-8'))
display(HTML("<h1>Intel Latest Vulnerabilities</h1>"))
count=0
urls=[]
for b in range(len(a)):
  if "intel-sa" in a[b]:
    urls.append("https://www.intel.com"+a[b][1:])
    count+=1
    if count == 10: 
        break
        
table="<table><tr><th>URL</th><th>Title</th><th>Severity</th><th>Last Date</th></tr>"
matrix=[]
for url in (urls):
  r = http.request('GET',url)
  a=re.findall('<title>(.*)</title>',r.data.decode('utf-8')) 
  b=re.findall('<meta name="lastModifieddate" content="(.*)\"',r.data.decode('utf-8')) 
  c=re.findall('<h2 class=" headline-font">(.*)</h2>',r.data.decode('utf-8')) 
  d=re.findall('<td>([C|H|M|L].*[H|L|W|M])</td>',r.data.decode('utf-8'),re.MULTILINE) 
  d[0]=d[0].replace("HIGH","3-HIGH")
  d[0]=d[0].replace("MEDIUM","2-MEDIUM")
  d[0]=d[0].replace("LOW","1-LOW")
  d[0]=d[0].replace("CRITICAL","4-CRITICAL")
  matrix.append([a[0],b[0],c[0],d[0],url])
  #print(r.data.decode('utf-8')) 
smatrix=sorted(matrix, key=lambda x: (x[3]),reverse=True)
for i in smatrix:
  table+="<tr><td><a href=\""+i[4]+"\">"+i[0]+"</a></td><td>"+i[2]+"</td><td>"+i[3]+"</td><td>"+i[1][:10]+"</td></tr>"
  
display(HTML(table+"</table>"))

URL,Title,Severity,Last Date
INTEL-SA-00325,Intel® VTune™ Amplifier for Windows* Advisory,3-HIGH,2020-01-07
INTEL-SA-00323,Intel® NUC® Firmware Advisory,3-HIGH,2019-12-05
INTEL-SA-00329,Intel® Processors Data Leakage Advisory,2-MEDIUM,2020-01-23
INTEL-SA-00314,Intel® Processor Graphics Advisory,2-MEDIUM,2020-01-13
INTEL-SA-00308,Intel® RWC 3 for Windows* Advisory,2-MEDIUM,2020-01-15
INTEL-SA-00306,Intel® Chipset Device Software Advisory,2-MEDIUM,2020-01-07
INTEL-SA-00300,Intel® SNMP Subagent Stand-Alone Advisory for Windows*,2-MEDIUM,2020-01-07
INTEL-SA-00324,Intel® RST Advisory,2-MEDIUM,2019-12-11
INTEL-SA-00317,Unexpected Page Fault in Virtualized Environment Advisory,2-MEDIUM,2019-12-09
INTEL-SA-00332,Intel® DAAL Advisory,1-LOW,2020-01-07


In [110]:
from datetime import date

today = date.today()
bdate_today = today.strftime("%b%Y")
date_today = (today.strftime("%b%Y")).lower()

http = urllib3.PoolManager()
url="https://www.oracle.com/security-alerts/cpu"+date_today+".html"
r = http.request('GET',url)

a=re.findall('Oracle Java SE,(.*)</a>',r.data.decode('utf-8'))
b=re.findall('id=\"AppendixJAVA\"(.*)<!-- END Oracle Java SE RISK MATRIX -->',r.data.decode('utf-8'),re.DOTALL)

display(HTML("<h1>Java "+bdate_today+" Vulnerabilites</h1>"))
print(url)
display(HTML("<h2>Affected Versions</h2>"))
display(HTML("Oracle Java SE,"+a[0]))
display(HTML("<h4>"+b[0][1:]))

https://www.oracle.com/security-alerts/cpujan2020.html


In [5]:
http = urllib3.PoolManager()
url="https://support.lenovo.com/gb/en/api/v4/search/psrss?language=en&country=gb&brand=EBG,TPG,IPG,MOTO,PHONE,TABLET,SMART"
r = http.request('GET',url)

a=re.findall('<link>http://support.lenovo.com/product_security/(\w+\d+)</link>',r.data.decode('utf-8'))
b=re.findall('<description><!\[CDATA\[([\w\s]+)\]\]\></description><link>',r.data.decode('utf-8'))
c=re.findall('<pubDate>(\w+, \d+ \w+ \d+ \d+:\d+:\d+ \w+)</pubDate>',r.data.decode('utf-8'))

count=0
urls=[]
table="<table><tr><th>Ref</th><th>Title</th><th>Date</th></tr>"
for d in range(len(a)):
  #print(b[c])
  if "PS" in a[d]:
    table+=("<tr><td><a href=\"http://support.lenovo.com/product_security/"+a[d]+"\">"+a[d]+"</a></td><td>"+b[d+1]+"</td><td>"+c[d][:16]+"</td></tr>")
    #print(table)
    count+=1
    if count == 5: 
        break

display(HTML("<h1>Lenovo Latest Vulnerabilities</h1>"))
display(HTML(table+"</table>"))

Ref,Title,Date
PS500303,Intel Processors Side Channel Data Leakage Vulnerabilities,"Tue, 28 Jan 2020"
PS500302,Intel Processor Graphics Vulnerability,"Fri, 10 Jan 2020"
PS500301,NVIDIA GeForce Experience Update,"Thu, 09 Jan 2020"
PS500300,Intel Chipset Device Software INF Utility Advisory,"Thu, 09 Jan 2020"
PS500295,Unexpected Page Fault in Virtualized Environment Advisory,"Tue, 03 Dec 2019"
